### Setup

In [1]:
""" Python Standard Library """
import os
import datetime as dt
import json
import logging
""" Third Party Imports """
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())
from covid_model.model import CovidModel
from covid_model.runnable_functions import do_single_fit, do_fit_scenarios, do_create_multiple_reports
from covid_model.utils import setup, get_filepath_prefix
from covid_model.analysis.charts import plot_transmission_control

os.environ['gcp_project'] = 'co-covid-models'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "co-covid-models-credentials.json"

/Users/emwu9912/PycharmProjects/covid-models


In [2]:
# set up the output directory for this Jupyter notebook
outdir = setup("20221109_scenario_projections.ipynb")

2022/11/08 14:42:23 INFO|============================================================


### Fit an initial scenario through February 2022

In [3]:
# designate the arguments for how the model will behave
model_args = {
    'params_defs': 'covid_model/analysis/20221109_nov_gov_briefing/20221109_scenario_params.json',
    'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/analysis/20221109_nov_gov_briefing/20221109_vacc_proj_params.json',
    'start_date': '2020-01-24',
    'end_date': '2024-01-01',
}

# this is how the fit will behave
# place the outdir argument here to tell the model fit where to go
fit_args = {'outdir': outdir,
            'fit_end_date': '2022-02-28'
}

# because all the scenarios are the same 
model = do_single_fit(**fit_args, **model_args)

2022/11/08 14:42:27 INFO|-------------------------|{} Retrieving vaccinations data
2022/11/08 14:42:28 INFO|-------------------------|{} Constructing vaccination projections
2022/11/08 14:42:30 INFO|-------------------------|{} Retrieving hospitalizations data
2022/11/08 14:42:32 INFO|----------------------|{} Prepping Model
2022/11/08 14:42:32 INFO|-----------------------|{} Prepping Model


KeyboardInterrupt: 

### Create and run scenarios from Feb 2022 to present

In [4]:
multiprocess = 3

scenario_params = json.load(open('covid_model/analysis/20221109_nov_gov_briefing/20221109_scenario_params.json'))

model_args = {
    'base_spec_id': 4296, # use the spec id that was output from the model fit
}
model_fit_args = {
    'outdir': outdir,
    'fit_start_date': '2022-03-01', # set the start date for the earliest point at which the scenarios start to differ from one another
    'pre_solve_model': True # force the model to establish initial conditions so the fit can start on the fit start date
}

# define vaccine effectiveness for < 5 (this is a multiplier for the baseline vaccine effectiveness for 0-19)
vacc_eff_lt5 = 0.5

# Create different scenarios to model
scenario_model_args = []
for (weak_escape, strong_escape) in [(0.6, 0.35), (0.5, 0.25), (0.7, 0.45)]:
    scrabble_seed = [{"param": "scrabble_seed", "attrs": None, "vals": {"2020-01-01": 0, "2022-09-15": 10,
                                                                       "2022-10-30": 0}}]
    hrf = {"2020-01-01": 1, "2022-03-01": (0.66 + 0.34*0.8),
           "2022-03-15": (0.34 + 0.66*0.8), "2022-03-30": 0.8}
    weak_param = [{"param": "immune_escape",
                   "from_attrs": {"immun": "weak",
                                  "variant": ["none", "wildtype", "alpha", "delta",
                                              "omicron", "ba2", "ba2121", "ba45"]},
                   "to_attrs": {"variant": ["scrabble"]},
                   "vals": {"2020-01-01": weak_escape},
                   "desc": "Scrabble variants immune escape value, weak immunity"}]
    strong_param = [{"param": "immune_escape",
                   "from_attrs": {"immun": "strong",
                                  "variant": ["none", "wildtype", "alpha", "delta",
                                              "omicron", "ba2", "ba2121", "ba45"]},
                   "to_attrs": {"variant": ["scrabble"]},
                   "vals": {"2020-01-01": strong_escape},
                   "desc": "Scrabble variants immune escape value, strong immunity"}]
    lt5_vacc_adjust = [{"param": "immunity",
                        "attrs": {'age': '0-19', 'vacc': 'shot1'},
                        "mults": {"2020-01-01": 1,
                                  "2022-06-24": 0.99 + 0.01*vacc_eff_lt5,
                                  "2022-06-30": 0.98 + 0.02*vacc_eff_lt5,
                                  "2022-07-08": 0.97 + 0.03*vacc_eff_lt5,
                                  "2022-07-19": 0.96 + 0.04*vacc_eff_lt5,
                                  "2022-07-29": 0.95 + 0.05*vacc_eff_lt5,
                                  "2022-08-11": 0.94 + 0.06*vacc_eff_lt5,
                                  "2022-08-30": 0.93 + 0.07*vacc_eff_lt5,
                                  "2022-09-26": 0.92 + 0.08*vacc_eff_lt5,
                                  "2022-10-26": 0.91 + 0.09*vacc_eff_lt5,},
                        "desc": "weighted average using share of 0-19 getting shot1 who are under 5"}]
    scenario_model_args.append({'params_defs': scenario_params + weak_param + strong_param + scrabble_seed + lt5_vacc_adjust,
                                'hosp_reporting_frac': hrf,
                                'tags': {'scrabble_escape_weak': weak_escape,
                                        'scrabble_escape_strong': strong_escape}})
            

In [5]:
# check how many scenarios there are
len(scenario_model_args)

3

In [6]:
# run the scenarios
models = do_fit_scenarios(base_model_args=model_args, scenario_args_list=scenario_model_args, fit_args=model_fit_args, multiprocess=multiprocess)

2022/11/08 14:44:49 INFO|============================================================
2022/11/08 14:44:49 INFO|============================================================
2022/11/08 14:44:49 INFO|============================================================
2022/11/08 14:44:53 INFO|-----------|{'scrabble_escape_weak': 0.6, 'scrabble_escape_strong': 0.35} Retrieving vaccinations data
2022/11/08 14:44:53 INFO|-----------|{'scrabble_escape_weak': 0.5, 'scrabble_escape_strong': 0.25} Retrieving vaccinations data
2022/11/08 14:44:53 INFO|-----------|{'scrabble_escape_weak': 0.7, 'scrabble_escape_strong': 0.45} Retrieving vaccinations data
2022/11/08 14:44:54 INFO|-----------|{'scrabble_escape_weak': 0.5, 'scrabble_escape_strong': 0.25} Constructing vaccination projections
2022/11/08 14:44:54 INFO|-----------|{'scrabble_escape_weak': 0.6, 'scrabble_escape_strong': 0.35} Constructing vaccination projections
2022/11/08 14:44:54 INFO|-----------|{'scrabble_escape_weak': 0.7, 'scrabble_escape_st

2022/11/08 14:47:25 INFO|--------|{'scrabble_escape_weak': 0.7, 'scrabble_escape_strong': 0.45, 'fit_batch': '3'}: Transmission control fit 4/7 completed in 12.393063957999999 seconds: {851: {'co': 0.8060404796394446}, 865: {'co': 0.8329794960850301}, 879: {'co': 0.8156065926405599}, 893: {'co': 0.8064824991900593}, 907: {'co': 0.8097232185803398}, 921: {'co': 0.7991453303900372}}
2022/11/08 14:47:25 INFO|---------|{'scrabble_escape_weak': 0.7, 'scrabble_escape_strong': 0.45, 'fit_batch': '3'}: Running forward sim
2022/11/08 14:47:26 INFO|--------|{'scrabble_escape_weak': 0.6, 'scrabble_escape_strong': 0.35, 'fit_batch': '3'}: Transmission control fit 4/7 completed in 12.425005749999997 seconds: {851: {'co': 0.8060404796394446}, 865: {'co': 0.8329794960850301}, 879: {'co': 0.8156065926405599}, 893: {'co': 0.8064824991900593}, 907: {'co': 0.8097232185803398}, 921: {'co': 0.7991453303900372}}
2022/11/08 14:47:26 INFO|---------|{'scrabble_escape_weak': 0.6, 'scrabble_escape_strong': 0.35,

2022/11/08 14:48:19 INFO|--------|{'scrabble_escape_weak': 0.6, 'scrabble_escape_strong': 0.35, 'fit_batch': '6', 'run_type': 'fit'}: fitted TC: {0: {'co': 0.8306366821816143}, 14: {'co': 0.6979993282832323}, 28: {'co': 0.5652815030779559}, 42: {'co': 1.3377630507629367e-15}, 56: {'co': 0.6966542156819525}, 70: {'co': 0.8332723113665885}, 84: {'co': 0.8296250664461475}, 98: {'co': 0.8316923553393842}, 112: {'co': 0.8774897769552777}, 126: {'co': 0.8243043563823643}, 140: {'co': 0.760452793097366}, 154: {'co': 0.6712880022306991}, 168: {'co': 0.8430551366399854}, 182: {'co': 0.7826363434875424}, 196: {'co': 0.8191511043640095}, 210: {'co': 0.7726071093170934}, 224: {'co': 0.7713215685714466}, 238: {'co': 0.7185439602724302}, 252: {'co': 0.7220971427071249}, 266: {'co': 0.6850776801451802}, 280: {'co': 0.7198400366509309}, 294: {'co': 0.7589414915394733}, 308: {'co': 0.8220709579032593}, 322: {'co': 0.8118910349627317}, 336: {'co': 0.7823429680786415}, 350: {'co': 0.7948870083003131}, 36

### Run the report for each fit model

In [7]:
# here you can also specify which variants you want to calculate immunity for
do_create_multiple_reports(models, multiprocess=multiprocess, outdir=outdir, prep_model=False, solve_model=True, immun_variants=['ba45', 'scrabble'], from_date='2022-01-01')

2022/11/08 14:49:40 INFO|============================================================
2022/11/08 14:49:40 INFO|--------|Solving model
2022/11/08 14:49:40 INFO|============================================================
2022/11/08 14:49:40 INFO|--------|Solving model
2022/11/08 14:49:41 INFO|============================================================
2022/11/08 14:49:41 INFO|--------|Solving model
2022/11/08 14:49:44 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/11/08 14:49:45 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/11/08 14:49:46 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [8]:
logging.info('Projecting')
for model in models:
    logging.info('')
    #model.prep()  # don't think we need to prep anymore.
    model.solve_seir()

    model.solution_sum_df(['seir', 'variant', 'immun']).unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_seir_variant_immun_total_all_at_once_projection.csv')
    model.solution_sum_df().unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_full_projection.csv')

    logging.info(f'{str(model.tags)}: Running forward sim')
    fig = plt.figure(figsize=(10, 10), dpi=300)
    ax = fig.add_subplot(211)
    hosps_df = model.modeled_vs_observed_hosps().reset_index('region').drop(columns='region')
    hosps_df.plot(ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2024-01-01', "%Y-%m-%d").date())
    ax = fig.add_subplot(212)
    plot_transmission_control(model, ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2024-01-01', "%Y-%m-%d").date())
    plt.savefig(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast.png')
    plt.close()
    hosps_df.to_csv(get_filepath_prefix(outdir, tags=model.tags) + '_model_forecast.csv')
    json.dump(model.tc, open(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast_tc.json', 'w'))

logging.info('Running reports')

2022/11/08 14:51:56 INFO|Projecting
2022/11/08 14:51:56 INFO|
2022/11/08 14:52:17 INFO|{'scrabble_escape_weak': 0.6, 'scrabble_escape_strong': 0.35, 'fit_batch': '6', 'run_type': 'fit'}: Running forward sim
2022/11/08 14:52:19 INFO|
2022/11/08 14:52:40 INFO|{'scrabble_escape_weak': 0.5, 'scrabble_escape_strong': 0.25, 'fit_batch': '6', 'run_type': 'fit'}: Running forward sim
2022/11/08 14:52:41 INFO|
2022/11/08 14:53:01 INFO|{'scrabble_escape_weak': 0.7, 'scrabble_escape_strong': 0.45, 'fit_batch': '6', 'run_type': 'fit'}: Running forward sim
2022/11/08 14:53:03 INFO|Running reports


In [ ]:
model.solution_ydf.to_csv('covid_model/output/20221109_scenario_projections.ipynb/state_model_solution_ydf.csv', index=False)